In [1]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.ndimage import binary_dilation
from skimage.feature import structure_tensor, structure_tensor_eigenvalues


In [2]:
def extract_patches_from_edges(image_path, gt_path, r=3):
    """
    Original: Canny→patches + binary labels from dilated red mask.
    """
    image_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image      = cv2.imread(image_path)
    gt_image   = cv2.imread(gt_path)

    # assume red mask on GT marks ridge
    red_mask = (gt_image[:,:,2] > 200) & (gt_image[:,:,1]<50) & (gt_image[:,:,0]<50)
    dilated_red_mask = binary_dilation(red_mask, np.ones((3,3)))

    edges = cv2.Canny(image_gray, 50, 200)

    h, w = image_gray.shape
    patches, labels = [], []
    for y in range(r, h-r):
        for x in range(r, w-r):
            if edges[y,x]==0: 
                continue
            lab = int(dilated_red_mask[y,x])
            p = image[y-r:y+r+1, x-r:x+r+1]
            if p.shape != (2*r+1,2*r+1,3): 
                continue
            patches.append(p)
            labels.append(lab)
    return np.array(patches), np.array(labels)

def load_full_dataset(image_dir, gt_dir):
    all_patches = []
    all_labels = []
    for i in tqdm(range(1, 81)):
        img_name = f"R_GImag{i:04d}.bmp"
        gt_name = f"GT_GoogleImage{i:03d}_Edge.bmp"
        image_path = os.path.join(image_dir, img_name)
        gt_path = os.path.join(gt_dir, gt_name)
        if os.path.exists(image_path) and os.path.exists(gt_path):
            patches, labels = extract_patches_from_edges(image_path, gt_path)
            all_patches.append(patches)
            all_labels.append(labels)

    all_patches = np.concatenate(all_patches, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Balance dataset
    pos_indices = np.where(all_labels == 1)[0]
    neg_indices = np.where(all_labels == 0)[0]
    n_samples = min(len(pos_indices), len(neg_indices))

    selected_pos = np.random.choice(pos_indices, n_samples, replace=False)
    selected_neg = np.random.choice(neg_indices, n_samples, replace=False)

    selected_indices = np.concatenate([selected_pos, selected_neg])
    np.random.shuffle(selected_indices)

    balanced_patches = all_patches[selected_indices]
    balanced_labels = all_labels[selected_indices]

    return balanced_patches, balanced_labels

def show_example_patches(patches, labels, label_filter=None, num=10):
    idxs = np.where(labels==label_filter)[0] if label_filter is not None else np.arange(len(patches))
    plt.figure(figsize=(10,2))
    for i,idx in enumerate(idxs[:num]):
        plt.subplot(1,num,i+1)
        plt.imshow(patches[idx])
        plt.title(labels[idx])
        plt.axis('off')
    plt.tight_layout(); plt.show()


In [3]:
# Cell 3 — Structure‑Tensor feature maps

def compute_structure_tensor_maps(gray, sigma=1.0):
    Jxx, Jxy, Jyy = structure_tensor(gray, sigma=sigma)
    ST = np.array([Jxx, Jxy, Jyy])
    λ1, λ2       = structure_tensor_eigenvalues(ST)
    strength    = np.sqrt(λ1)
    coherence   = (λ1 - λ2) / (λ1 + λ2 + 1e-12)
    orientation = 0.5 * np.arctan2(2*Jxy, Jxx - Jyy)
    return strength, coherence, orientation


In [4]:
def extract_patches_with_ST(image_path, gt_path, r=3, bucket=False, bins=None):
    """
    Returns:
      patches:   (N, (2r+1)^2*3 + (bucket? one-hot:3) ) float32
      labels:    (N,) {0,1}
    """
    image_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)/255.0
    image      = cv2.imread(image_path)
    gt_image   = cv2.imread(gt_path)
    # same red mask logic
    red_mask = (gt_image[:,:,2]>200)&(gt_image[:,:,1]<50)&(gt_image[:,:,0]<50)
    red_mask = binary_dilation(red_mask, np.ones((3,3)))
    edges = cv2.Canny((image_gray*255).astype(np.uint8), 50,200)>0

    S, C, O = compute_structure_tensor_maps(image_gray)
    h,w = image_gray.shape

    feats, labs = [], []
    for y in range(r, h-r):
      for x in range(r, w-r):
        if not edges[y,x]: continue
        lab = int(red_mask[y,x])
        patch = image[y-r:y+r+1, x-r:x+r+1].astype(np.float32)/255.0
        flat  = patch.reshape(-1)  # ( (2r+1)^2 * 3, )
        s,c,o = S[y,x], C[y,x], O[y,x]
        if bucket:
          sb = np.digitize(s, bins['strength'])-1
          cb = np.digitize(c, bins['coherence'])-1
          ob = np.digitize(o, bins['orientation'])-1
          s_feat = np.eye(len(bins['strength'])-1)[sb]
          c_feat = np.eye(len(bins['coherence'])-1)[cb]
          o_feat = np.eye(len(bins['orientation'])-1)[ob]
          f = np.concatenate([flat, s_feat, c_feat, o_feat])
        else:
          f = np.concatenate([flat, [s,c,o]])
        feats.append(f); labs.append(lab)

    return np.stack(feats).astype(np.float32), np.array(labs).astype(np.float32)


In [5]:
def make_loader(X, y, bs=64):
    ds = TensorDataset(torch.from_numpy(X), torch.from_numpy(y).unsqueeze(1))
    n = len(ds); t = int(0.8*n)
    tr, te = random_split(ds, [t, n-t])
    return DataLoader(tr, batch_size=bs, shuffle=True), DataLoader(te, batch_size=bs, shuffle=False)


In [6]:
class RidgeClassifier(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, 16)
        self.fc3 = nn.Linear(16, 1)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        return torch.sigmoid(self.fc3(x))

In [7]:
image_dir = "web_dataset/images"
gt_dir    = "web_dataset/ground_truth"
patch_size=7  # 2*r+1
r = patch_size//2

# 1) ORIGINAL
orig_patches_list, orig_labels_list = [], []
for i in range(1, 81):
    img_name = f"R_GImag{i:04d}.bmp"
    gt_name  = f"GT_GoogleImage{i:03d}_Edge.bmp"
    ip = os.path.join(image_dir, img_name)
    gp = os.path.join(gt_dir,   gt_name)
    if os.path.exists(ip) and os.path.exists(gp):
        p, l = extract_patches_from_edges(ip, gp, r=r)
        orig_patches_list.append(p)
        orig_labels_list.append(l)

orig_patches = np.concatenate(orig_patches_list, axis=0)
orig_labels  = np.concatenate(orig_labels_list,  axis=0)
X_orig = orig_patches.reshape(len(orig_patches), -1).astype(np.float32)/255.0
y_orig = orig_labels.astype(np.float32)

# 2) ST-CONTINUOUS
st_feats_list, st_labels_list = [], []
for i in range(1, 81):
    img_name = f"R_GImag{i:04d}.bmp"
    gt_name  = f"GT_GoogleImage{i:03d}_Edge.bmp"
    ip = os.path.join(image_dir, img_name)
    gp = os.path.join(gt_dir,   gt_name)
    if os.path.exists(ip) and os.path.exists(gp):
        f, l = extract_patches_with_ST(ip, gp, r=r, bucket=False)
        st_feats_list.append(f)
        st_labels_list.append(l)

X_st = np.concatenate(st_feats_list, axis=0)
y_st = np.concatenate(st_labels_list, axis=0)

# Define bins based on ST continuous features
bins = {
  'strength':    np.linspace(0, np.max(X_st[:,-3]), 6+1),
  'coherence':   np.linspace(0, 1.0,               3+1),
  'orientation': np.linspace(-np.pi/2, np.pi/2,   16+1),
}

# 3) ST-BUCKETED
bkt_feats_list, bkt_labels_list = [], []
for i in range(1, 81):
    img_name = f"R_GImag{i:04d}.bmp"
    gt_name  = f"GT_GoogleImage{i:03d}_Edge.bmp"
    ip = os.path.join(image_dir, img_name)
    gp = os.path.join(gt_dir,   gt_name)
    if os.path.exists(ip) and os.path.exists(gp):
        f, l = extract_patches_with_ST(ip, gp, r=r, bucket=True, bins=bins)
        bkt_feats_list.append(f)
        bkt_labels_list.append(l)

X_bkt = np.concatenate(bkt_feats_list, axis=0)
y_bkt = np.concatenate(bkt_labels_list, axis=0)

# Create DataLoaders
loader_orig = make_loader(X_orig, y_orig)
loader_st   = make_loader(X_st,   y_st)
loader_bkt  = make_loader(X_bkt,  y_bkt)

print("Original:", X_orig.shape, y_orig.shape)
print("ST-Continuous:", X_st.shape, y_st.shape)
print("ST-Bucketed:", X_bkt.shape, y_bkt.shape)

Original: (7138070, 147) (7138070,)
ST-Continuous: (7138070, 150) (7138070,)
ST-Bucketed: (7138070, 172) (7138070,)


In [8]:
print(torch.version.cuda, torch.cuda.is_available())

12.6 True


In [ ]:
def train_eval(loader, in_dim, lr=3e-3, epochs=5):
    train_loader, test_loader = loader
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Training on {device}.\n")
    model = RidgeClassifier(in_dim).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.BCELoss()
    for ep in range(epochs):
        model.train()
        for xb,yb in tqdm(train_loader, desc=f"Ep{ep}"):
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            loss = loss_fn(model(xb), yb)
            loss.backward()
            opt.step()
        # eval
        model.eval()
        tot, cnt, acc = 0.,0,0
        with torch.no_grad():
            for xb,yb in test_loader:
                xb,yb = xb.to(device), yb.to(device)
                out = model(xb)
                tot += loss_fn(out,yb).item()*len(xb)
                cnt += len(xb)
                pred = (out>0.5).float()
                acc += (pred==yb).sum().item()
        print(f"  Val Loss: {tot/cnt:.4f}, Acc: {acc/cnt:.3f}")
    return model

# run all three
print(">>> ORIGINAL")
m0 = train_eval(loader_orig, X_orig.shape[1])

print(">>> ST CONTINUOUS")
m1 = train_eval(loader_st,   X_st.shape[1])

print(">>> ST BUCKETED")
m2 = train_eval(loader_bkt,  X_bkt.shape[1])


>>> ORIGINAL
Training on cuda.



Ep0: 100%|██████████| 89226/89226 [03:58<00:00, 373.82it/s]


  Val Loss: 0.0158, Acc: 0.994


Ep1: 100%|██████████| 89226/89226 [05:22<00:00, 276.45it/s]


  Val Loss: 0.0146, Acc: 0.995


Ep2:  31%|███▏      | 27921/89226 [01:50<04:00, 255.29it/s]